# User Settings

In [1]:
# What folder do you want to save the results in?
# Individual CSV's for each coin saved with coin name as filename
RESULTS_FOLDER = 'results'

# Setup Notebook

In [2]:
!pip install fastcore
!pip install aiohttp


     |████████████████████████████████| 56 kB 2.4 MB/s 
     |████████████████████████████████| 1.1 MB 5.0 MB/s 
     |████████████████████████████████| 160 kB 53.9 MB/s 
     |████████████████████████████████| 271 kB 55.7 MB/s 
     |████████████████████████████████| 192 kB 66.9 MB/s 


In [3]:
from fastprogress import progress_bar
import requests 
from bs4 import BeautifulSoup
import pandas as pd
import re
import humanize
from IPython.display import clear_output, display
from pathlib import Path
import numpy as np
from fastcore.all import *

import asyncio
import aiohttp

# this is to allow it to run in google colab environment
import nest_asyncio
nest_asyncio.apply()


In [4]:
Path(RESULTS_FOLDER).mkdir(exist_ok=True, parents=True)

#Helper Functions

In [54]:
def parse_strlist(sl):
    clean = re.sub("[\[\],\s]","",sl)
    splitted = re.split("[\'\"]",clean)
    values_only = [s for s in splitted if s != '']
    return values_only

def get_bitinfochart_graph_values(url, var_name):
  response = requests.get(url)
  soup = BeautifulSoup(response.text, 'html.parser')

  scripts = soup.find_all('script')
  for script in scripts:
      if 'd = new Dygraph(document.getElementById("container")' in script.text:
          StrList = script.text
          StrList = '[[' + StrList.split('[[')[-1]
          StrList = StrList.split(']]')[0] +']]'
          StrList = StrList.replace("new Date(", '').replace(')','')
          dataList = parse_strlist(StrList)

  date = []
  value = []
  for each in dataList:
      if (dataList.index(each) % 2) == 0:
          date.append(each)
      else:
          value.append(each)

  df = pd.DataFrame(list(zip(date, value)), columns=["date",var_name])
  return df


In [55]:
# This function takes a list of df's then merge's them on the date field with an outer join

def merge_dfs(df_list):
  df_merged = None
  for i in range(len(df_list)-1):
    if i == 0:
      df_merged = df_list[i].merge(df_list[i+1], on='date', how='outer')
    else:
      df_merged = df_merged.merge(df_list[i+1], on='date', how='outer')

  return df_merged


# Generate List of Pages to Scrape

In [59]:
# Manually copy/pasted to use as a starting point

chart_dict_list = [
                    {'url': 'https://bitinfocharts.com/comparison/tweets-btc.html', 'name': 'tweets'},
                    {'url': 'https://bitinfocharts.com/comparison/google_trends-btc.html', 'name': 'google_trends'},

                    ]

In [60]:
# Get list of all available coins

url = 'https://bitinfocharts.com/comparison/tweets.html'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

coin_dict_list = []


for span in soup.find_all('span'):
  
  if 's_coins' in str(span.get('class')):
    name = span.get('title').lower()
    coin = span.get('data-coin')
    coin_dict_list.append({'full_name': name,
                'coin': coin})

coin_dict_list[:3]

[{'coin': 'btc', 'full_name': 'bitcoin'},
 {'coin': 'eth', 'full_name': 'ethereum'},
 {'coin': 'busd', 'full_name': 'busd'}]

In [70]:
# Combine the list of coins with the list of url's to create a master dictionary with coins and url's to scrape

for coin_dict in coin_dict_list:
  coin_dict['scrape_details'] = []
  for chart_dict in chart_dict_list:
    temp_dict = chart_dict.copy()
    
    url = temp_dict['url']
    url = url.replace('bitcoin', coin_dict['full_name'])
    url = url.replace('btc', coin_dict['coin'])
    url = url.replace(' ', '%20')

    temp_dict['url'] = url
    coin_dict['scrape_details'].append(temp_dict)

coin_dict_list[:2]

[{'coin': 'btc',
  'full_name': 'bitcoin',
  'scrape_details': [{'name': 'tweets',
    'url': 'https://bitinfocharts.com/comparison/tweets-btc.html'},
   {'name': 'google_trends',
    'url': 'https://bitinfocharts.com/comparison/google_trends-btc.html'}]},
 {'coin': 'eth',
  'full_name': 'ethereum',
  'scrape_details': [{'name': 'tweets',
    'url': 'https://bitinfocharts.com/comparison/tweets-eth.html'},
   {'name': 'google_trends',
    'url': 'https://bitinfocharts.com/comparison/google_trends-eth.html'}]}]

# Async Helpers

In [63]:
async def fetch(session, url, full_name, coin, var_name):

  try:
    async with session.get(url) as resp:
        return await resp.text(), full_name, coin, var_name
  except:
    return 'failed', full_name, coin, var_name
        # Catch HTTP errors/exceptions here


In [64]:
async def fetch_concurrent(coin_dictionary):
  page_results = []
  loop = asyncio.get_event_loop()


  async with aiohttp.ClientSession() as session:
    tasks = []
    for coin_dict in coin_dictionary:
      for page in coin_dict['scrape_details']:

        tasks.append(loop.create_task(fetch(session, 
                                            page['url'], 
                                            coin_dict['full_name'], 
                                            coin_dict['coin'],
                                            page['name'])))

    for result in asyncio.as_completed(tasks):
      text, full_name, coin, var_name = await result
      result_dict = {'response_text': text,
                     'full_name': full_name,
                     'coin': coin,
                     'var_name': var_name}
      page_results.append(result_dict)
            
      
    return page_results



In [65]:
def extract_info_from_response(response_dict):

  response_text = response_dict['response_text']
  var_name = response_dict['var_name']
  
  soup = BeautifulSoup(response_text, 'html.parser')

  scripts = soup.find_all('script')
  for script in scripts:
      if 'd = new Dygraph(document.getElementById("container")' in script.text:
          StrList = script.text
          StrList = '[[' + StrList.split('[[')[-1]
          StrList = StrList.split(']]')[0] +']]'
          StrList = StrList.replace("new Date(", '').replace(')','')
          dataList = parse_strlist(StrList)

  date = []
  value = []
  for each in dataList:
      if (dataList.index(each) % 2) == 0:
          date.append(each)
      else:
          value.append(each)

  df = pd.DataFrame(list(zip(date, value)), columns=["date",var_name])


  if df[var_name].str.isnumeric().any():
    if df[var_name].str.contains('nullnull').any():
      df[var_name] = np.nan

    return df
  
  else:
    return pd.DataFrame(columns=['date', var_name])


# Async Code Run

- Takes about 13s for all the requests
- then takes 6 minutes to convert everything to dataframes

In [66]:
%%time
http_responses = asyncio.run(fetch_concurrent(coin_dict_list))
len(http_responses)

CPU times: user 5.16 s, sys: 589 ms, total: 5.74 s
Wall time: 11.6 s


In [67]:
coin_dict['full_name']

'bitcoinz'

In [68]:
coin_dict

{'coin': 'btcz',
 'full_name': 'bitcoinz',
 'scrape_details': [{'name': 'tweets',
   'url': 'https://bitinfocharts.com/comparison/tweets-btcz.html'},
  {'name': 'google_trends',
   'url': 'https://bitinfocharts.com/comparison/google_trends-btcz.html'}]}

In [71]:
coin_merged_df_list = []

for coin_dict in progress_bar(coin_dict_list):
  print(f"Now Processing - {coin_dict['full_name']}")
  coin_df_list = []
  for response in progress_bar(http_responses):
    if coin_dict['full_name'] == response['full_name']:
      coin_df_list.append(extract_info_from_response(response))
  
  coin_df = merge_dfs(coin_df_list)
  coin_df['full_name'] = coin_dict['full_name']
  coin_df['coin'] = coin_dict['coin']
  coin_merged_df_list.append(coin_df)

  file_path = RESULTS_FOLDER + '/' +coin_dict['full_name'] + '__' + coin_dict['coin'] + '.csv'
  coin_df.to_csv(file_path)

  clear_output()

      
combined_df = pd.concat(coin_merged_df_list, ignore_index=True, sort=False)

combined_df.to_csv('all_bitcoininfochart_data.csv')
combined_df.to_pickle('all_bitcoininfochart_data.pkl')

In [72]:
print('Creating Zip File...')
!zip -r -q results.zip {RESULTS_FOLDER}
print('Complete!')

Creating Zip File...
Complete!
